In [21]:
# coding=utf-8
import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin
from collections import Counter
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [3]:
def fit(X,y):
        cnt = Counter()
        for i in range(X.shape[1]):
            for j in range(np.max(X[:, i])+1):
                indices = X[:, i] == j
                if np.sum(indices) > 0:
                    val = y[indices].mean()
                else:
                    val = y.mean()
                cnt[(i, j)] = val
                
        return cnt
    
def transform(X, cnt):
    X_new = np.copy(X)
    for i in range(X.shape[1]): 
        for j in range(np.max(X[:, i])+1):
            indices = X[:, i] == j
            if np.sum(indices) > 0:
                X_new[indices, i] = cnt[(i, j)]
    return X_new

In [358]:
class CustomTransformer(TransformerMixin):
    
    def __init__(self, alpha):
        self.alpha = alpha
    
    def fit(self, X, y):
        self.cnt = Counter()
        for i in range(X.shape[1]):
            for j in range(np.max(X[:, i])+1):
                indices = X[:, i] == j
                if np.sum(indices) > 0:
                    val = y[indices].mean()
                else:
                    val = y.mean()
                self.cnt[(i, j)] = val
        
        self.global_mean = y.mean()
        
        return self
    
#     def transform(self, X):
#         X_new = np.copy(X)
#         for i in range(X.shape[1]): 
#             for j in range(np.max(X[:, i])+1):
#                 indices = X[:, i] == j
#                 if np.sum(indices) > 0:
#                     X_new[indices, i] = self.cnt[(i, j)]
#         return X_new

    def transform(self, X):
        X_new = np.copy(X)
        for i in range(X.shape[1]):
            for j in range(np.max(X[:,i])+1):
                indices = X[:, i] == j
                if np.sum(indices) > 0:
                     X_new[indices, i] = 
                        #self.cnt[(i, j)]
                        
                        #(self.cnt[(i, j)]*X.shape[0] + \  
                        #self.global_mean*self.alpha) / (X.shape[0] + self.alpha)
                        
                       # self.
        return X_new

In [368]:
X.shape

(8124, 22)

In [359]:
df = pd.read_csv('mushrooms.csv', header=None)

In [360]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [361]:
df.shape

(8124, 23)

In [362]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])

label_encoder = LabelEncoder()

for i in range(X.shape[1]):
    X[:, i] = label_encoder.fit_transform(X[:, i])
    
y = np.equal(y, 'p').astype(int)
X_data, y_data = X, y

In [363]:
col_num = 5
len(np.unique(X_data[:,col_num])) == len(np.unique(df.loc[:,col_num+1].values))

True

In [545]:
lr_params = {'penalty':'l2',
             'C': 0.0077,
             'random_state': 10}
  #{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’},            
             #'class_weight': 'balanced'}

In [546]:
pipeline = make_pipeline(
        CustomTransformer(alpha=0),
        LogisticRegression(**lr_params)
    )

In [547]:
pipeline.steps[0]

('customtransformer', <__main__.CustomTransformer at 0x1dcb1fce400>)

In [548]:
print( "Score:" , round(cross_val_score(pipeline, X_data, y_data, cv=3).mean(),4))

Score: 0.9318


In [550]:
30*61.9*0.87

1615.59